# Big Data y Machine Learning (UBA) 2025
## Clase 6 - Análisis de Componentes Principales (PCA)

Componentes principales (PCA, en inglés) es una técnica de **aprendizaje no supervisado**. Es decir que nos encontramos en una situación donde tenemos información de un conjunto de variables o features ($X_1, X_2, ..., X_p$), pero no sobre una variable de resultado o outcome ($Y$). Vamos a tratar de ajustar algoritmos que interpreten la distribución de nuestros datos y encuentren relaciones interesantes entre éstos, trabajando con la naturaleza propia de los datos y sin un outcome de interés $Y$.  
Esto se diferencia del **aprendizaje supervisado**, caso en el cual los estimadores se usan para **predecir** resultados basados en datos que poseen un outcome o variable de resultado $Y$ (puede ser una etiqueta -clasificación- o un valor -regresión-).

Los algoritmos de aprendizaje no supervisado pueden ser muy útiles para casos en los que se busca **reducir la dimensionalidad**, por ejemplo cuando se busca visualizar datos de gran dimensionalidad o se busca crear un índice. PCA suele emplearse como parte del **análisis descriptivo y exploratorio de datos**.

Supongamos que tenemos $n$ observaciones y $p$ variables y queremos visualizarlas como parte de una análisis exploratorio de los datos.

Podríamos realizar gráficos de a 2 variables, pero serían muchos si $p$ es grande...
Entonces vamos a buscar una representación de los datos en menos dimensiones (2 usualmente) que capture la mayor información posible.

Las dimensiones serán combinaciones lineales de las $p$ variables que tienen la mayor varianza posible.

Vamos a trabajar con la librería [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)

### Ejemplo 1

In [ ]:
#Instalamos los paquetes necesarios
!pip install statsmodels
!pip install scikit-learn
!pip install ISLP

# Alternativa
#import sys
#!{sys.executable} -m pip install statsmodels
#!{sys.executable} -m pip install scikit-learn
#!{sys.executable} -m pip install ISLP

In [ ]:
import ISLP
from ISLP import load_data
from statsmodels.datasets import get_rdataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
import os
print(os.getcwd())

In [ ]:
#Descargar los archivos de esta clase en una carpeta y poner el directorio
os.chdir("/Users/mnromero/Dropbox/COURSES/2025 - S1- Big Data y Machine Learning (UBA)/Clases/Clase 6_PCA") 

Vamos a trabajar con una base de datos que provee el [libro ISLP](https://islp.readthedocs.io/en/latest/datasets/USArrests.html).

##### Violent Crime Rates by US State
Contiene información sobre arrestos por asaltos, asesinatos y violaciones cada 100.000 habitantes en 50 estados de Estados Unidos en 1973. También tiene información sobre el porcentaje de población viviendo en zonas urbanas.

- Murder: Murder arrests (per 100,000)
- Assault: Assault arrests (per 100,000)
- Rape: Rape arrests (per 100,000)
- UrbanPop: Percent urban population

#### 1. Chequeamos la base de datos a usar

In [ ]:
arrests = get_rdataset('USArrests').data
print(arrests.shape)
print("\n", arrests.info)
print("\n", arrests.dtypes)
print("\n", arrests.head())

In [ ]:
# Estadistica descriptiva (promedio) de las variables originales
print(arrests.mean())

##### 1.2 Transformamos las variables
Escalamos las variables antes de usar PCA

In [ ]:
# Inicializamos el transformador, 
scaler = StandardScaler(with_std=True, with_mean=True) 
# Aplicamos fit_transform al DataFrame
arrests_transformed = pd.DataFrame(scaler.fit_transform(arrests), columns=arrests.columns)

In [ ]:
# Chequeamos que tengan media 0 y desvio estandar 1
print("Promedio luego de la transformación\n",arrests_transformed.mean()) # luego de la estandarización la media es cero
print("Desvío estandár luego de la transformación\n",arrests_transformed.std()) # la desviación estandar es uno

In [ ]:
# Visualizamos
print(arrests_transformed.head())

Importante: Por definición, `PCA()` requiere centrar las variables para que tengan media cero pero las escalamos por eficiencia computacional y no alterar los pesos de las variables.

### 2. Aplicamos PCA 
Estamos buscando maximizar la varianza de los predictores con la restricción de normalización

In [ ]:
# Ajustamos el modelo
pca = PCA()
arrests_pca = pca.fit_transform(arrests_transformed)

#### 2.1 Veamos los distintos elementos de dicho método no supervisado

In [ ]:
# Scores/ Indices (z_im) donde i (filas) m (componentes)
scores = arrests_pca
print(scores)

In [ ]:
# Loadings vectors
loading_vectors = pca.components_ # cada fila corresponde a un CP y cada columna, a una variable
print("Loadings:\n", pca.components_)
print("Loadings del CP1:\n",pca.components_[0]) 
pca.components_[0,0] #loadings del CP1 variable 1


#### 2.2 Recordemos la norma igual 1 de la suma de los ponderadores al cuadrado 

In [ ]:
# Notar que si tomamos los loadings/ponderadores del primer componente principal, por ejemplo:
(0.53589947)**2+(0.58318363)**2+(0.27819087)**2+(0.5434309)**2 
# La suma de sus cuadrados vemos que es igual a 1. Es la restricción que habíamos puesto!

Tarea probar que: $\phi_1'*\phi_2=0$

### 3. Grafico de Dispersión (*Biplot*)
Un ventaja de este método no supervisado es enfocarnos en los primeros dos componentes que nos resumen gran parte de la información original de nuestra matriz X.

In [ ]:
i, j = 0, 1 # Componentes
fig, ax = plt.subplots(1, 1, figsize=(5, 5)) # creamos 1 subplot
ax.scatter(scores[:,0], scores[:,1]) # graficamos los valores de los CP1 y CP2
ax.set_xlabel('ComponentePrincipal%d' % (i+1))
ax.set_ylabel('ComponentePrincipal%d' % (j+1))
plt.title('Analisis de los primeros dos componentes principales')
for k in range(pca.components_.shape[1]): # loop que itera por la cantidad de features
    ax.arrow(0, 0, pca.components_[i,k], pca.components_[j,k]) # flecha desde el origen (0) a las coordenadas
    ax.text(pca.components_[i,k], pca.components_[j,k], arrests.columns[k]) # al final de cada flecha, nombre de la variable

In [ ]:
# Biplot
# Ajustes, extendemos longitud de las flechas e invertimos el eje y

i, j = 0, 1 # Componentes

scale_arrow = s_ = 2 # para extender la longitud de las flechas y que se vean mejor
scores[:,1] *= -1
pca.components_[1] *= -1 # gira el eje y (CP2)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.scatter(scores[:,0], scores[:,1]) 
ax.set_xlabel('ComponentePrincipal%d' % (i+1))
ax.set_ylabel('ComponentePrincipal%d' % (j+1))
plt.title('Analisis de los primeros dos componentes principales')
for k in range(pca.components_.shape[1]):
    ax.arrow(0, 0, s_*pca.components_[i,k], s_*pca.components_[j,k])
    ax.text(s_*pca.components_[i,k], s_*pca.components_[j,k], arrests.columns[k])

#### 4. Proporcion de la Varianza explicada
Para entender cuánta información de la matriz original $X$ resume cada uno de los componentes, podemos calcular y visualizar la varianza de la matriz original $X$ explicada por cada uno de los componentes.

In [ ]:
# % de la Varianza explicada por los componentes 
print(pca.explained_variance_ratio_) # CP1 explica el 62% de la varianza

In [ ]:
%%capture 
fig, axes = plt.subplots(1, 2, figsize=(10, 4)) # 2 subplots uno al lado del otro
ticks = np.arange(pca.n_components_)+1 # para crear ticks en el eje horizontal
ax = axes[0]
ax.plot(ticks, pca.explained_variance_ratio_ , marker='o')
ax.set_xlabel('Componente principal');
ax.set_ylabel('Prop. de la varianza explicada')
ax.set_ylim([0,1])
ax.set_xticks(ticks)
# capture suprime la visualización de la figura parcialmente terminada

In [ ]:
ax = axes[1]
ax.plot(ticks, pca.explained_variance_ratio_.cumsum(), marker='o') 
ax.set_xlabel('Componente principal')
ax.set_ylabel('Suma acumulada de la varianza explicada')
ax.set_ylim([0, 1])
ax.set_xticks(ticks)
fig

Para decidir *qué número de componentes usar*, podemos consultar un scree plot que nos muestre la proporción de variable explicada para cada uno de los componentes y la variación en la varianza total explicada por el total de los componentes.

Típicamente se elige la cantidad de componentes para la cual la proporción de la varianza explicada cae para cada componente principal adicional (cuando hay un codo en el scree plot)

### Ejemplo 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

Vamos a trabajar con un dataset de vinos (de scikit-learn). Contiene características de 178 vinos y a qué segmento de consumidores pertenecen

In [ ]:
# Importar el dataset y breve exploración
wine_data = pd.read_csv('Wine.csv')
print(wine_data.shape)
print(wine_data.dtypes)
print(wine_data.head())

In [ ]:
# Separamos datos entre X e Y (por ahora, haremos de cuenta que no contamos con Y)
wine_features = wine_data.iloc[:, 0:13].values
wine_customer_segment = wine_data.iloc[:, 13].values

# Vemos las etiquetas posibles de customer segment
wine_customer_segment_unique, counts = np.unique(wine_customer_segment, return_counts=True)
for value, count in zip(wine_customer_segment_unique, counts):
    print(f"Value: {value}, Count: {count}")

In [ ]:
# Preprocesamiento. Estandarizar las variables
# Iniciar scaler y aplicarlo
sc = StandardScaler()
wine_features_transformed = sc.fit_transform(wine_features)

Por qué estandarizamos? El análisis es sensible a la varianza de las variables originales y eso puede ocasionar problemas a la hora de elegir los CPs

In [ ]:
# Aplicar PCA

pca = PCA(n_components = 2)

wine_pca = pca.fit_transform(wine_features_transformed) # Obtenemos los scores

In [ ]:
# Scores
wine_scores = wine_pca

# % de la Varianza explicada por los componentes
print("Varianza explicada:", pca.explained_variance_ratio_)
# El primer componente principal explica el 36% de la varianza, mientras que el segundo, explica el 19%

# Loading vectors
loading_vectors = pca.components_ # cada fila corresponde a un CP y cada columna, a una variable
print("Loadings:\n", pca.components_)
print("Loadings del CP1:\n",pca.components_[0]) 

# Visualizamos features y loadings
for i, loading_vector in enumerate(loading_vectors):
    print(f"\nLoading Vector CP{i+1}:")
    for j, feature in enumerate(wine_data.columns[:-1]):
        print(f"{feature}: {round(loading_vector[j],3)}")
    print()

In [ ]:
# Crear un DataFrame para los componentes principales
pca_df = pd.DataFrame(data=wine_pca, columns=['Componente_1', 'Componente_2'])

# Añadir la variable objetivo al DataFrame de los componentes principales
pca_df['Customer_Segment'] = wine_customer_segment
pca_df

In [ ]:
# Graficamos los componentes
plt.figure(figsize=(6, 6))
plt.scatter(pca_df['Componente_1'], pca_df['Componente_2'], c=wine_data['Customer_Segment'], cmap='viridis')
plt.xlabel('Componente Principal 1', fontsize=11)
plt.ylabel('Componente Principal 2', fontsize=11)
plt.title('Analisis de los primeros dos componentes principales')
plt.colorbar(label='Customer Segment')
plt.show()

In [ ]:
 # Otra forma de graficar
plt.figure(figsize=(6, 6))
plt.xlabel('Componente Principal 1', fontsize=11)
plt.ylabel('Componente Principal 2', fontsize=11)
plt.title("Wine Data",fontsize=16)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)

targets = [1, 2, 3]
colors = ['red', 'green', 'blue']

for target, color in zip(targets,colors):
    indices_graf = pca_df['Customer_Segment'] == target
    plt.scatter(pca_df.loc[indices_graf, 'Componente_1'], pca_df.loc[indices_graf, 'Componente_2'], c = color, s = 50)

#plt.xlim(-4,4)
#plt.ylim(-4,4)
plt.legend(targets)

La representación bidimensional de los datos tridimensionales capta correctamente el patrón principal de los datos: las observaciones rojas, azules y verdes, siguen estando en la representación bidimensional. 

Nota: aquí usamos dos componentes pero podríamos haber usado 1 o más de 2. 